In [14]:
import pandas as pd 
import numpy as np

# Multiline Printing

In [1]:
from timeit import default_timer as timer
import time

In [11]:
start = timer()
for i in range(1, 11):
    time.sleep(1)
    time_elapsed = timer() - start
    time_per_iteration = time_elapsed / i
    estimated_time = (10 - i) * time_per_iteration
    print(f"""Time elapsed: {round(time_elapsed, 2)} seconds.\
    {round(100 * (i / 10), 2)}% complete. \
    Estimated time remaining: {round(estimated_time, 2)} seconds.""", end = '\r')

# MultiThreading

In [12]:
import threading
from queue import Queue
from timeit import default_timer as timer

In [15]:
test = pd.read_csv('input/test.csv', 
                  usecols = ['key', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 
                             'dropoff_longitude'])
test_rounded = test.round(3).groupby(['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'])['key'].\
                            count().reset_index().\
                            drop(columns = 'key')

len(test_rounded)

9870

In [16]:
test_rounded['pickup'] = test_rounded['pickup_latitude'].astype(str) + "," + test_rounded['pickup_longitude'].astype(str)
test_rounded['dropoff'] = test_rounded['dropoff_latitude'].astype(str) + "," + test_rounded['dropoff_longitude'].astype(str)
test_rounded['l'] = test_rounded['pickup'] + "|" + test_rounded['dropoff']

In [20]:
def row_proc(l):
    pickup, dropoff = l.split('|')
    geocode_result = gmaps.distance_matrix(pickup,dropoff)
    #print (geocode_result)
    try:
        distance = float(geocode_result['rows'][0]['elements'][0]['distance']['text'].split()[0])
        duration = geocode_result['rows'][0]['elements'][0]['duration']['text'].split()
        
        if len(duration)==4:
            mins = float(duration[0])*60 + float(duration[2])
        
        elif len(duration) > 4:
            print(geocode_result)
            
        else:
            mins = float(duration[0])
    except:
        mins = np.nan
        distance = np.nan
        
    try:
        origin = str(geocode_result['origin_addresses'][0])
        dest = str(geocode_result['destination_addresses'][0])
    
    except:
        origin = np.nan
        distance = np.nan
        
    return (distance, mins, origin, dest)

In [45]:
import googlemaps

with open('/home/ec2-user/keys/gmap.txt', 'r') as f:
    gmap_key = str(f.read()).strip()
    
# Authenticate with google maps
gmaps = googlemaps.Client(key=gmap_key)

In [25]:
start = timer()

print_lock = threading.Lock()

l_list = list(test_rounded['l'])

# Create a new queue
l_queue = Queue()

# List to hold results
run_l = []
distances = []
durations = []
origins = []
dests = []
TRACKER = 0

# Function to take an element from the queue and execute task
def process_queue():
    while True:
        global TRACKER
        TRACKER += 1
        # Get next element from the queue
        current_l = l_queue.get()
        # Record the coordinates
        run_l.append(current_l)
        
        # Run the function
        r = row_proc(current_l)
        
        # Record the results
        distances.append(r[0])
        durations.append(r[1])
        origins.append(r[2])
        dests.append(r[3])
        
        if TRACKER % 1000 == 0:
            time_elapsed = timer() - start
            time_per_record = time_elapsed / TRACKER 
            estimated_time = ((len(test_rounded) - TRACKER) * time_per_record) / 3600
            print(f'Time elapsed: {round(time_elapsed, 2)} seconds.\
            {round(100 * (TRACKER / len(test_rounded)), 2)}% complete.\
            Estimated time remaining: {round(estimated_time, 2)} hours.', end = '\r')
            
        # Signal to the queue that the task is done
        l_queue.task_done()

# Start 100 threads
for i in range(100):
    t = threading.Thread(target = process_queue)
    # Set daemon to be true
    t.daemon = True
    t.start()
    
# Put each element of the list on the queue
for current_l in l_list:
    l_queue.put(current_l)

# Execture the queue
l_queue.join()

end = timer()
print(f'{round(end - start, 2)} seconds elapsed.')

86.15 seconds elapsed.onds.            91.19% complete.            Estimated time remaining: 0.0 hours.urs.Time elapsed: 59.75 seconds.            70.92% complete.            Estimated time remaining: 0.01 hours.Time elapsed: 59.75 seconds.            70.92% complete.            Estimated time remaining: 0.01 hours.s.Time elapsed: 42.72 seconds.            50.66% complete.            Estimated time remaining: 0.01 hours.Time elapsed: 42.72 seconds.            50.66% complete.            Estimated time remaining: 0.01 hours.Time elapsed: 51.22 seconds.            60.79% complete.            Estimated time remaining: 0.01 hours.Time elapsed: 68.33 seconds.            81.05% complete.            Estimated time remaining: 0.0 hours.


Exception in thread Thread-4672:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-24-2291fc5140c0>", line 44, in process_queue
    l_queue.task_done()
  File "/opt/conda/lib/python3.6/queue.py", line 68, in task_done
    raise ValueError('task_done() called too many times')
ValueError: task_done() called too many times
Exception in thread Thread-5049:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-25-7fc2e743121e>", line 44, in process_queue
    l_queue.task_done()
  File "/opt/conda/lib/python3.6/queue.py", line 68, in task_done
    raise ValueError('task_done() ca

In [26]:
from multiprocessing.dummy import Pool as ThreadPool

In [ ]:
def row_proc_process(l):
    

In [44]:
from tqdm import tqdm_notebook

pool = ThreadPool(100)
results = pool.map(row_proc, test_rounded['l'][:1000])
pool.close()

KeyboardInterrupt: 

In [41]:
results

0it [00:04, ?it/s]